# API Center ❤️ all APIs

## Import from Apigee API Management
![image](../../images/import-from-gcp.png)

Playground to experiment importing APIs from Apigee. We start by creating an API Center instance using [Bicep resource definition](https://learn.microsoft.com/en-us/azure/templates/microsoft.apicenter/services?pivots=deployment-language-bicep) and then we will use the [GCP CLI](https://cloud.google.com/sdk/docs/install-sdk) to export the Apigee APIs and the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest) to import the APIs into API Center.

💡 Every step outlined below leverages the power of PowerShell scripts. This is designed so you can seamlessly integrate them into your automation workflows, CI/CD pipelines, webhooks, and more.

### Learning Objectives - Upon completing this lab, you should be able to:
- Deploy Azure API Center using Bicep.
- Import environments from Apigee
- Import APIs from Apigee
- Import API deployments from Apigee
- Search and discover the APIs registered in API Center.

### Prerequisites
- Install or update to the latest [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download)
- Install or update to the latest [Visual Studio Code](https://code.visualstudio.com/)
- Install the [Polyglot Notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) from the VS Code marketplace
- Install or update to the latest [PowerShell](https://learn.microsoft.com/en-us/powershell/scripting/install/installing-powershell)
- Install or update to the latest [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli). The Azure API Center extension will automatically install the first time you run an [az apic](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest) command.
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)
- Install or update to the latest [GCP CLI](https://cloud.google.com/sdk/docs/install-sdk) and set your Apigee project


### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- Adjust the APIC location parameter according your preferences and [region availability.](https://learn.microsoft.com/en-us/azure/api-center/overview#available-regions) 


In [ ]:

$deploymentName = Split-Path -Path (Get-Location) -Leaf
$resourceGroupName = "lab-$deploymentName" # change the name to match your naming style
$resourceGroupLocation = "westeurope"
$apicResourceNamePrefix = "apic"
$apicResourceSku = "free"
$apicResourceTags = @{
  "lab" = $deploymentName
}

# the following metadata will be created during the Bicep deployment
$metadata = @(
  @{
    "name" = 'createdAt'
    "schema" = '{"title":"createdAt","description":"When the API was created in Apigee","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'deployStartTime'
    "schema" = '{"title":"deployStartTime","description":"When the API was deployed in Apigee","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'deployment'
        "required" = $false
      }
    )
  }
)

# the following environment(s) will be created during the deployment
$environments = @(
)

Write-Output "✅ Variables initialized ⌚ $(Get-Date -Format 'HH:mm:ss')"


### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [ ]:
$resourceGroupOutput = az group create --name $resourceGroupName --location $resourceGroupLocation

if ($LASTEXITCODE -ne 0) {
    Write-Output $resourceGroupOutput
} else {
    Write-Output "✅ Azure Resource Grpup $resourceGroupName created ⌚ $(Get-Date -Format 'HH:mm:ss')"
}

### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
$bicepParameters = @{
    "`$schema" = "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#"
    "contentVersion" = "1.0.0.0"
    "parameters" = @{
        "apicResourceNamePrefix" = @{ "value" = $apicResourceNamePrefix }
        "apicResourceTags" = @{ "value" = $apicResourceTags }
        "apicMetadataSchema" = @{ "value" = $metadata }
        "apicEnvironments" = @{ "value" = $environments }
    }
}

$bicepParametersJson = ConvertTo-Json -InputObject $bicepParameters -Depth 10
Set-Content -Path "params.json" -Value $bicepParametersJson

# Execute the Azure CLI command to create the deployment
az deployment group create --name $deploymentName --resource-group $resourceGroupName --template-file "main.bicep" --parameters "params.json"



### 3️⃣ Get the deployment outputs

We will set the `apicResourceName` variable with the value that was returned from the deployment 

In [ ]:
$deploymentOutput = az deployment group show --name $deploymentName -g $resourceGroupName --query properties.outputs.apicResourceName.value -o json
if ($LASTEXITCODE -ne 0) {
    Write-Output $deploymentOutput
} else {
    $apicResourceName = $deploymentOutput | ConvertFrom-Json
}
Write-Output "👉🏻 API Center name: $apicResourceName"

### 4️⃣ Get and import Apigee environments

We will start by [getting the environments from Apigee](https://cloud.google.com/sdk/gcloud/reference/apigee) and create the environments in API Center


In [ ]:

$apigeeEnvironmentsJson = gcloud apigee environments list --format=json
$apigeeEnvironments = $apigeeEnvironmentsJson | ConvertFrom-Json
foreach ($environment in $apigeeEnvironments) {
    Write-Output "👉🏻 Creating environment $environment"
    az apic environment create -g $resourceGroupName -s $apicResourceName --environment-id $environment --title $environment --type "production" `
            --server '{\"type\":\"Apigee API Management\"}'
}


### 5️⃣ Import the Apigee APIs into API Center

Get the APIs from Apigee and register in API Center with version and definition.

In [ ]:
$apigeeAPIsJson = gcloud apigee apis list --format=json
$apigeeAPIs = $apigeeAPIsJson | ConvertFrom-Json
foreach ($apigeeAPI in $apigeeAPIs) {
    $apiJson = gcloud apigee apis describe $apigeeAPI --format=json
    $api = $apiJson | ConvertFrom-Json

    $apiCustomProperties = @{
        "createdAt" = $api.metaData.createdAt
    }

    $apiName = $api.name
    if ($api.metaData.subType -eq "PROXY") {
        $apiCustomPropertiesJson = ConvertTo-Json -InputObject $apiCustomProperties -Depth 10 -Compress
        write-output "👉🏻 Creating API $apiName"
        az apic api create -g $resourceGroupName -s $apicResourceName --api-id $apiName --type REST `
            --title $apiName  `
            --custom-properties $apiCustomPropertiesJson.replace('"','\"')
        foreach ($revision in $api.revision) {
            write-output "👉🏻 Creating version $revision of API $apiName"
            az apic api version create -g $resourceGroupName -s $apicResourceName --api-id $apiName --version-id "v-$revision" --title "Version $revision" `
                        --lifecycle-stage "production"

            write-output "👉🏻 Creating definition for version $revision of API $apiName"
            az apic api definition create -g $resourceGroupName -s $apicResourceName --api-id $apiName `
                --version-id "v-$revision" --definition-id "openapi" --title "OpenAPI" --description "OpenAPI spec"      

        }
    }
}



### 6️⃣ Import the Apigee API Deployments into API Center

Get the Deployments from Apigee and create them in API Center

In [ ]:
$apigeeDeploymentsListJson = gcloud apigee deployments list --format=json
$apigeeDeploymentsList = $apigeeDeploymentsListJson | ConvertFrom-Json
foreach ($apigeeDeployment in $apigeeDeploymentsList) {
    $apiName = $apigeeDeployment.apiProxy
    $envName = $apigeeDeployment.environment
    $apigeeDeploymentJson = gcloud apigee deployments describe --api=$apiName --environment=$envName --format=json

    $apigeeDeployment = $apigeeDeploymentJson | ConvertFrom-Json

    $deploymentCustomProperties = @{
        "deployStartTime" = $apigeeDeployment.deployStartTime
    }

    $deploymentCustomPropertiesJson = ConvertTo-Json -InputObject $deploymentCustomProperties -Depth 10 -Compress
    $revision = $apigeeDeployment.revision
    write-output "👉🏻 Creating deployment for version $revision of API $apiName"
    az apic api deployment create -g $resourceGroupName -s $apicResourceName --deployment-id "$apiName-deployment" `
            --title "$apiName Deployment"  --api-id $apiName `
            --environment-id "/workspaces/default/environments/$envName" `
            --definition-id "/workspaces/default/apis/another-petstore/versions/v-$revision/definitions/openapi" `
            --server '{\"runtimeUri\":[\"\"]}' `
            --custom-properties $deploymentCustomPropertiesJson.replace('"','\"')
}

### 7️⃣ Discover the APIs that were just imported

You can discover the APIs with fhe following methods:
- With the Azure Portal
- With the [self-hosted API Center Portal](https://learn.microsoft.com/en-us/azure/api-center/enable-api-center-portal)
- With the [VS Code extension](https://learn.microsoft.com/en-us/azure/api-center/use-vscode-extension-copilot) that is integrated with GitHub Copilot Chat.
- With the CLI, the service REST API and more

Here we will use the [list command](https://learn.microsoft.com/en-us/cli/azure/apic/api?view=azure-cli-latest#az-apic-api-list) to display the APIs that we have just imported. 


In [ ]:
az apic api list -g $resourceGroupName -s $apicResourceName --query "[].{Name:name, Title:title, Kind:kind, ContactEmail:contacts[0].email}" -o table

### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.